## Задание 2. SQL

In [5]:
# импортируем нужные библиотеки
import pandahouse as ph

In [9]:
# для подключения к таблице default в clickhouse:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'}

### 2.1 Очень усердные ученики.

2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

2.1.2 Задача

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.



#### Решение

Перед запросом я проверила за какой период представлены данные в таблице, так как нужно получить результат именно за текущий месяц. Так как данные представлены только за один месяц, то в основном запросе корректировка не требуется, тем более в задании не сказано что такое текущий месяц. 

Запрос который использовала для проверки месяца:
- select min(timest), max(timest), st_id
- from peas  
- group by st_id

In [10]:
# Оптимальный запрос о количестве учеников

# пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
with students as (   
    select st_id, count(timest)
    from peas
    where correct = 1
    group by st_id
    having count(timest) > 19)
    
select count(st_id) from students
"""
students_1 = ph.read_clickhouse(query, connection=connection_default)
students_1

,count(st_id)
0,136


### 2.2 Оптимизация воронки

2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

2.2.2 Задача

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

#### Решение

Определим некоторые метрики: 
- ARPU считается, как средний доход с привлечённого пользователя
- ARPAU считается, как средний доход с привлечённого активного пользователя
- CR в покупку считается, как отношение пользователей купивших полный доступ к общему числу пользователей (остальные CR считаются так же, но с дополнительной корректировкой, обозначенной в условии). 

In [11]:
# пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
WITH
         for_ARPU  as (
            select test_grp, sum(money)/count(distinct(st_id)) as ARPU
            from studs left join default.final_project_check on studs.st_id = final_project_check.st_id
            group by test_grp), 
         for_CR as (
            select test_grp, (countIf(distinct(final_project_check.st_id), final_project_check.st_id != '') 
            /count(distinct(st_id)))* 100 as CR
            from studs left join default.final_project_check on studs.st_id = final_project_check.st_id
            group by test_grp), 
         for_CR_active as (
            select test_grp, (countIf(distinct(final_project_check.st_id), final_project_check.st_id != '') 
            /count(distinct(st_id)))* 100 as CR_active
            from studs left join default.final_project_check on studs.st_id = final_project_check.st_id
            where st_id in (select st_id
                          from peas
                           where correct = 1
                          group by st_id
                           having count(timest) > 10)
            group by test_grp),
         for_ARPAU as (
            select test_grp, sum(money) /count(distinct(st_id)) as ARPAU
            from studs left join default.final_project_check on studs.st_id = final_project_check.st_id
            where st_id in (select st_id
                           from peas
                           where correct = 1
                          group by st_id
                          having count(timest) > 10)
            group by test_grp),
          for_CR_active_math as (
            select test_grp, (countIf(distinct(final_project_check.st_id), subject = 'Math') 
            /count(distinct(st_id)))* 100 as CR_active_math
            from studs left join default.final_project_check on studs.st_id = final_project_check.st_id
            where st_id in (select st_id               
                           from peas
                          where subject = 'Math' and correct = 1
                         group by st_id
                         having count(timest) >= 2)
            group by test_grp)


select for_ARPU.test_grp as test_group, ARPU, ARPAU, CR, CR_active, CR_active_math
from for_ARPU join for_CR on for_ARPU.test_grp = for_CR.test_grp
                join for_CR_active on for_CR.test_grp = for_CR_active.test_grp 
                join for_ARPAU on for_CR_active.test_grp = for_ARPAU.test_grp 
                join for_CR_active_math on for_ARPAU.test_grp =  for_CR_active_math.test_grp
"""
students_2 = ph.read_clickhouse(query, connection=connection_default)
students_2

,test_group,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,4540.983607,10393.700787,4.918033,11.023622,6.122449
1,pilot,11508.474576,29739.583333,10.847458,26.041667,9.523810


На основе метрик можно сказать, что новый экран оплаты увеличит конверсию и доход компании. 